<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Long-Short-Term-Memory/blob/main/LSTM_6I1O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.random.set_seed(777)

df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/The2_MAX.txt')
print("Train.shape : ",df_train.shape)
train_data = df_train.iloc[:,:]
print(train_data)


In [ ]:
def MinMaxScaler(data) : 
  denom = np.max(data,0) - np.min(data,0)
  nume = data - np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min) : 
  diff = max - min
  back = data * diff + min
  return back


In [ ]:
seqLength = 3
dataDim = df_train.shape[1]
outputDim = 1
hiddenDim = 100
iterations = 10
lr = 0.001
batch_size = 64

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
trainSet_for_result = trainSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)

trainSet = MinMaxScaler(trainSet)

print(trainSet)
print(trainSet.shape)

In [ ]:
trainSet_1 = trainSet.iloc[:60000, :]
testSet_1 = trainSet.iloc[60000:, :]

def sliding_window(time_series, seq_length) : 
  dataX = []
  dataY = []
  for i in range(0, len(time_series) - seq_length) : 
    _x = time_series.iloc[i:i + seq_length, : -1]
    _y = time_series.iloc[i+ seq_length, -1]
    dataX.append(_x)
    dataY.append(_y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet_1, seqLength)
testX, testY = sliding_window(testSet_1, seqLength)

print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = 6
output_dim = 1
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh', input_shape = (seqLength, input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh', return_sequences = True, kernel_initializer = initializer))
model.add(layers.Dense(output_dim, activation = 'sigmoid'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer = adam, loss = 'mse')
model.summary()


In [ ]:
print("train X : ",trainX.shape)
print("train Y : ",trainY.shape)
print("input_dim : ",input_dim)

In [ ]:
hist = model.fit(trainX, trainY, epochs = 5, batch_size = batch_size)

In [ ]:
xhat = testX
yhat = model.predict(xhat)

print("xhat : ",xhat.shape)
print("yhat : ",yhat.shape)

print("testY : ",testY.shape)

In [ ]:
xhat = testX
xhat_train = trainX
yhat = model.predict(xhat)
yhat_train = model.predict(xhat_train)

print("Estimated.shape : ",yhat.shape, type(yhat))
print("Real.shape : ", testY.shape, type(testY))

plt.figure()
plt.plot(yhat[:len(yhat)], label = "Estimated by Stacked LSTM")
plt.plot(testY[:len(testY)], label = "Real")
plt.legend(prop = {'size' : 10})

print(yhat)